In [24]:
import pandas as pd
import google.generativeai as genai
from dotenv import load_dotenv
import os


In [25]:

# Charger les variables d'environnement
load_dotenv()

# Configuration sécurisée
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

if not GEMINI_API_KEY:
    raise ValueError("❌ Clé API manquante ! Vérifie ton fichier .env")

genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel('gemini-2.5-flash')


# CALCUL DES STATISTIQUES

In [26]:
df = pd.read_csv('donnees_utilisateur.csv')

# Déterminer si le joueur a gagné, perdu ou fait nul
def determine_result(row):
    if row['winner'] == 'TIE':
        return 'Nul'
    elif row['winner'] == row['team']:
        return 'Victoire'
    else:
        return 'Défaite'

df['Résultat'] = df.apply(determine_result, axis=1)

# Stats globales
stats = {
    'totalGames': len(df),
    'wins': len(df[df['Résultat'] == 'Victoire']),
    'losses': len(df[df['Résultat'] == 'Défaite']),
    'ties': len(df[df['Résultat'] == 'Nul']),
}

stats['winrate'] = round((stats['wins'] / stats['totalGames']) * 100, 2)

# Stats par rôle
stats['roleDistribution'] = df['role'].value_counts().to_dict()

# Winrate par rôle
def calculate_winrate_by_role(df, role):
    role_games = df[df['role'] == role]
    if len(role_games) == 0:
        return 0
    wins = len(role_games[role_games['Résultat'] == 'Victoire'])
    return round((wins / len(role_games)) * 100, 2)

stats['winrateByRole'] = {
    'DEF': calculate_winrate_by_role(df, 'DEF'),
    'ATK': calculate_winrate_by_role(df, 'ATK')
}

# Buts moyens par rôle (en retirant les buts contre son camp)
df['buts_nets'] = df['player_goals'] - df['player_own_goals']

stats['goalsByRole'] = {
    'DEF': round(df[df['role'] == 'DEF']['buts_nets'].mean(), 2),
    'ATK': round(df[df['role'] == 'ATK']['buts_nets'].mean(), 2),
    'overall': round(df['buts_nets'].mean(), 2)
}

stats['totalGoals'] = int(df['player_goals'].sum())
stats['totalOwnGoals'] = int(df['player_own_goals'].sum())
stats['totalSaves'] = int(df['player_saves'].sum())
stats['avgSaves'] = round(df['player_saves'].mean(), 2)
stats['avgGoals'] = round(df['buts_nets'].mean(), 2)


# GÉNÉRATION DU CONSEIL AVEC GEMINI

In [27]:
prompt = f"""
Tu es un coach professionnel de Baby-foot. Analyse mes statistiques et donne 1 conseil concret pour augmenter mes chances de victoire au prochain match :

📊 MES STATISTIQUES :
- Matchs joués : {stats['totalGames']}
- Winrate global : {stats['winrate']}%
- Victoires : {stats['wins']} | Défaites : {stats['losses']} | Nuls : {stats['ties']}

🎯 PAR RÔLE :
- Défenseur : {stats['roleDistribution'].get('DEF', 0)} matchs (WR: {stats['winrateByRole']['DEF']}%, {stats['goalsByRole']['DEF']} buts/match)
- Attaquant : {stats['roleDistribution'].get('ATK', 0)} matchs (WR: {stats['winrateByRole']['ATK']}%, {stats['goalsByRole']['ATK']} buts/match)

⚽ PERFORMANCES :
- Buts totaux : {stats['totalGoals']}
- Arrêts totaux : {stats['totalSaves']} (moy: {stats['avgSaves']}/match)

EXEMPLES DE RÉPONSE ATTENDUE:
- "Tu gagnes plus de matchs quand tu joues en défense (statistique). Privilégie ce rôle"
- "Tes statisques montrent un ratio de but/match plus élevé quand tu joues l'équipe bleu." 

Ne donne que le conseil et les statistiques sur lesquelles tu t'appuies. Sois concis et efficace.
"""

response = model.generate_content(prompt)
conseil = response.text


# EXPORT EN TXT

In [28]:
with open('conseils_gemini.txt', 'w', encoding='utf-8') as f:
    f.write("=" * 50 + "\n")
    f.write("CONSEILS D'AMÉLIORATION\n")
    f.write("=" * 50 + "\n\n")
    f.write(conseil)
    f.write("\n\n" + "=" * 50 + "\n")
    f.write("Statistiques analysées :\n")
    f.write(f"- {stats['totalGames']} matchs\n")
    f.write(f"- Winrate : {stats['winrate']}%\n")

print("✅ Analyse terminée ! Fichier 'conseils_gemini.txt' créé.")
print("\n📊 STATS :")
print(stats)
print("\n💡 CONSEILS :")
print(conseil)


✅ Analyse terminée ! Fichier 'conseils_gemini.txt' créé.

📊 STATS :
{'totalGames': 91, 'wins': 41, 'losses': 45, 'ties': 5, 'winrate': 45.05, 'roleDistribution': {'DEF': 59, 'ATK': 32}, 'winrateByRole': {'DEF': 45.76, 'ATK': 43.75}, 'goalsByRole': {'DEF': 2.66, 'ATK': 2.28, 'overall': 2.53}, 'totalGoals': 232, 'totalOwnGoals': 2, 'totalSaves': 465, 'avgSaves': 5.11, 'avgGoals': 2.53}

💡 CONSEILS :
Privilégie le rôle de défenseur. Ton winrate est plus élevé dans ce rôle (45.76% contre 43.75% en attaque).
